In [ ]:
from fractions import Fraction

from phase_gadgets import *

In [ ]:
pg1 = PhaseGadgetForm(qubits=6, phase_gadgets=14)\
    .set_all_connections_random(probability=0.3)\
    .set_many_connections(5, {0,1,2,3,4,5,6})

circ1 = pg1.to_extracted_circ()

# Split the last input's gadget into parts (eg. {0123456} → {0}, {1234}, {56}).
# Splitting with semantic equality requires adding an odd number of lines.
pg2 = PhaseGadgetForm(qubits=8, phase_gadgets=14)\
    .set_all_connections_random(probability=0.3)\
    .set_many_connections(5, {0})\
    .set_many_connections(6, {1,2,3,4})\
    .set_many_connections(7, {5,6})\
    .set_copies(5, {6,7})

circ2 = pg2.to_extracted_circ();

zx.draw(circ1)
print(circ1.stats(depth=True))
zx.draw(circ2)
print(circ2.stats(depth=True))

In [ ]:
zx.draw(pg1.to_graph())
zx.draw(pg2.to_graph(do_copies = True), labels=True)

In [ ]:
# Apparantly my laptop is weak and can't allocate 64gb. Tsk.
# Smaller sanity-check test case it is.
pg1 = PhaseGadgetForm(qubits=4, phase_gadgets=7)\
    .set_all_connections_random(probability=0.5)\
    .set_many_connections(3, {0,1,2,3,4,5})

circ1 = pg1.to_extracted_circ()

pg2 = PhaseGadgetForm(qubits=6, phase_gadgets=7)\
    .set_all_connections_random(probability=0.5)\
    .set_many_connections(3, {0})\
    .set_many_connections(4, {1,2,3,4})\
    .set_many_connections(5, {5})\
    .set_copies(3, {4,5})

circ2 = pg2.to_extracted_circ();

zx.draw(circ1)
print(circ1.stats(depth=True))
zx.draw(circ2)
print(circ2.stats(depth=True))

In [ ]:
g1 = pg1.to_graph()
g2 = pg2.to_graph(do_copies = True)
zx.draw(g1)
zx.draw(g2)
print(f"And are these equal? {'Yes!' if zx.compare_tensors(g1,g2) else 'Nooooooo...'}")

In [ ]:
zx.draw(g1, labels=True)
from graph_helpers import *
og1 = OpenGraph(g1)
# These types of diagrams always have gflow.
# (XY-plane) g[input] = {corresponding outputs}
# (YZ-plane) g[phase gadget] = {gadget} ∪ {outputs corresponding to 𝒩(gadget)}
# all non-outputs ≺ all outputs; arbitrary within those two groups
(g,d) = find_max_delayed_flow(og1)
print(f"Correction sets g: {g}")
print(f"Depths 0: {[q for q in d.keys() if d[q] == 0]}")
print(f"Depths 1: {[q for q in d.keys() if d[q] == 1]}")
print(f"Depths 2: {[q for q in d.keys() if d[q] == 2]}")

In [ ]:
import pyzx.simplify as simplify
import pyzx.rules as rules

def pivot(g, uv, draw=False):
    iters = simplify.simp(
        g,
        name = "Single pivot",
        match = rules.match_pivot_parallel,
        rewrite = rules.pivot,
        matchf = lambda e : uv[0] in e and uv[1] in e,
        quiet = True
    )
    if iters != 1:
        raise ValueError("ew")
    if draw:
        zx.draw(g, labels=True)

def lcomp(g, u, draw=False):
    iters = simplify.simp(
        g,
        name = "Single pivot",
        match = rules.match_lcomp_parallel,
        rewrite = rules.lcomp,
        matchf = lambda v : u == v,
        quiet = True
    )
    if iters != 1:
        raise ValueError("ew")
    if draw:
        zx.draw(g, labels=True)

def cnot_outputs(g, src, dst, draw=False):
    src_outputs = [v for v in g.neighbors(src) if g.edge_type((src,v)) == EdgeType.SIMPLE]
    dst_outputs = [v for v in g.neighbors(dst) if g.edge_type((dst,v)) == EdgeType.SIMPLE]
    if len(src_outputs) != 1 != len(dst_outputs):
        raise ValueError("Malformed graph; src or dst aren't linked to a single output.")
    
    src_neighbours = set(g.neighbors(src)).difference(src_outputs)
    dst_neighbours = set(g.neighbors(dst)).difference(dst_outputs)
    
    # src's neighbours are now symmetrically diff'd with dst's
    g.remove_edges([(src, n) for n in src_neighbours])
    g.add_edges([(src, n) for n in src_neighbours.symmetric_difference(dst_neighbours)], EdgeType.HADAMARD)

    # add the cnot
    cnot_src = insert_vertex(g, (src, src_outputs[0]), VertexType.Z, phase=0, move_vertices=True)
    cnot_dst = insert_vertex(g, (dst, dst_outputs[0]), VertexType.X, phase=0, move_vertices=False)
    g.add_edges([(cnot_src, cnot_dst)], EdgeType.SIMPLE)
    
    if draw:
        zx.draw(g, labels=True)

In [ ]:
pg3 = PhaseGadgetForm(qubits=3, phase_gadgets=3)\
    .set_many_connections(0, {0,1})\
    .set_many_connections(1, {0,2})\
    .set_many_connections(2, {1,2})

circ3 = pg3.to_extracted_circ();

zx.draw(pg3.to_graph())
zx.draw(circ3)
print(circ3.stats(depth=True))

In [ ]:
g3 = pg3.to_graph()
#g3.add_edge((5,14), EdgeType.HADAMARD)
zx.draw(g3, labels=True)
insert_vertex(g3, (0,1), VertexType.Z, phase=0, move_vertices = True, extra_hadamards = True)
insert_vertex(g3, (4,5), VertexType.Z, phase=0, move_vertices = False, extra_hadamards = True)
insert_vertex(g3, (8,9), VertexType.Z, phase=0, move_vertices = False, extra_hadamards = True)
insert_vertex(g3, (1,2), VertexType.Z, phase=0, move_vertices = True)
insert_vertex(g3, (5,6), VertexType.Z, phase=0, move_vertices = False)
insert_vertex(g3, (9,10), VertexType.Z, phase=0, move_vertices = False)
zx.draw(g3, labels=True)
# Remove the extracted part in order not to confuse pyzx
g3.remove_vertices([2,3,6,7,10,11])
for i in [21,22,23]:
    g3.set_type(i, VertexType.BOUNDARY)
g3.set_outputs([21,22,23])
zx.draw(g3, labels=True)
pivot(g3, (1,12), draw=True)
# Removes CZ between frontier verts, the double LC had, and the T gate
g3.remove_vertex(12)
g3.add_edge((13,21), EdgeType.SIMPLE)
zx.draw(g3, labels=True)
g3.set_phase(13, 0)
zx.draw(g3, labels=True)
g3.set_position(13, 2, 2)
zx.draw(g3, labels=True)

In [ ]:
pg3 = PhaseGadgetForm(qubits=3, phase_gadgets=3)\
    .set_many_connections(0, {0,1})\
    .set_many_connections(1, {0,2})\
    .set_many_connections(2, {1,2})\
    .set_many_phases([Fraction(1,4), Fraction(3,4), Fraction(5,4)])
pg4 = PhaseGadgetForm(qubits=3, phase_gadgets=2)\
    .set_many_connections(0, {0})\
    .set_many_connections(1, {0,1})\
    .set_many_connections(2, {0,1})\
    .set_many_phases([Fraction(3,4), Fraction(5,4)])
g3 = pg3.to_graph()
g4 = pg4.to_graph()

from pyzx import Circuit
circ_pre = Circuit(3)
circ_pre.add_gate(zx.gates.CNOT(1,0))

circ_post = Circuit(3)
circ_post.add_gate(zx.gates.XPhase(0, Fraction(1,4)))
circ_post.add_gate(zx.gates.CNOT(0,1))

g4 = circ_pre.to_graph() + g4 + circ_post.to_graph()

zx.draw(g3)
zx.draw(g4)
zx.compare_tensors(g3,g4)

In [ ]:
circ = Circuit(3)
circ.add_gate(zx.gates.CNOT(2,0))
circ.add_gate(zx.gates.CNOT(2,1))
circ.add_gate(zx.gates.HAD(0))
circ.add_gate(zx.gates.HAD(1))
circ.add_gate(zx.gates.HAD(2))
circ.add_gate(zx.gates.XPhase(1,Fraction(5,4)))
circ.add_gate(zx.gates.XPhase(0,Fraction(3,4)))
circ.add_gate(zx.gates.CNOT(1,2))
circ.add_gate(zx.gates.CNOT(0,1))
circ.add_gate(zx.gates.CNOT(0,2))
circ.add_gate(zx.gates.XPhase(0,Fraction(1,4)))
circ.add_gate(zx.gates.CNOT(0,1))
zx.draw(g3)
zx.draw(circ)
zx.compare_tensors(g3,circ.to_graph())

In [ ]:
circ = Circuit(3)
circ.add_gate(zx.gates.CNOT(2,0))
circ.add_gate(zx.gates.CNOT(2,1))
circ.add_gate(zx.gates.HAD(0))
circ.add_gate(zx.gates.HAD(1))
circ.add_gate(zx.gates.HAD(2))
circ.add_gate(zx.gates.HAD(1))
circ.add_gate(zx.gates.ZPhase(1,Fraction(5,4)))
circ.add_gate(zx.gates.HAD(1))
circ.add_gate(zx.gates.HAD(0))
circ.add_gate(zx.gates.ZPhase(0,Fraction(3,4)))
circ.add_gate(zx.gates.HAD(0))
circ.add_gate(zx.gates.CNOT(1,2))
circ.add_gate(zx.gates.CNOT(0,1))
circ.add_gate(zx.gates.CNOT(0,2))
circ.add_gate(zx.gates.HAD(0))
circ.add_gate(zx.gates.ZPhase(0,Fraction(1,4)))
circ.add_gate(zx.gates.HAD(0))
circ.add_gate(zx.gates.CNOT(0,1))
circ = zx.optimize.basic_optimization(circ.to_basic_gates(), do_swaps=True).to_basic_gates()
zx.draw(circ)
zx.compare_tensors(g3,circ.to_graph())
print(circ.stats(depth=True))